In [32]:
import pandas as pd
import glob
import os

# Dataframe zur Analyse
os.chdir = '/bundestag_klimaschutz/'
all_files = glob.glob("*.csv")

file_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    file_list.append(df)

df = pd.concat(file_list, axis=0, ignore_index=True)

In [35]:
df.drop('Jens Ackermann;"FDP";1975;"Sachsen-Anhalt";;"2. Juli 1975";"http://www.jens-ackermann.info";"jens.ackermann@bundestag.de"', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop(df.index[:1], inplace=True)

In [45]:
import re

# Fakten zum Datensatz
anzahl_sitzungen = len(all_files)
anzahl_reden = len(df.index)

# anzahl wörter (alle Wörter)
df_new = df[df['inhalt_neu'].notnull()]

def count_words(speech):
    words = speech.split()
    word_count = len(words)
    return word_count

dfc = df_new.copy()
dfc['word_count'] = dfc.apply(lambda row: count_words(row['inhalt_neu']), axis=1)

words_total = dfc['word_count'].sum()

# Anzahl des Wortes "Klimawandel"
def count_climate(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    result = re.findall("klimawandel", string)
    return len(result)

def count_any_climate_regex(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    resultKlima = re.findall(r"klima\w+", string)
    resultUmwelt = re.findall(r"umwelt\w+", string)#noch genauer suchen
    return len(resultKlima) + len(resultUmwelt)

def count_klimahysterie(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    result = re.findall("klimahysterie", string)
    return len(result)


dfc['Anzahl Klimawandel'] = dfc.apply(lambda row: count_climate(row['inhalt_neu']), axis=1)
dfc['Anzahl Klimahysterie'] = dfc.apply(lambda row: count_klimahysterie(row['inhalt_neu']), axis=1) 

climate_total = dfc['Anzahl Klimawandel'].sum() 
klimahysterie_total = dfc['Anzahl Klimahysterie'].sum()

def count_any_climate(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    result = re.findall("klima", string) #noch genauer suchen
    return len(result)
  
dfc['Anzahl Klima/Umwelt'] = dfc.apply(lambda row: count_any_climate_regex(row['inhalt_neu']), axis=1) 
#any_climate_total = dfc['Anzahl Klima'].sum()

In [4]:
n2_df = dfc.loc[dfc['Anzahl Klimahysterie'] != 0]

In [46]:
import numpy as np

n3_df = dfc.loc[dfc['Anzahl Klima/Umwelt'] != 0]
print(n3_df.head())
len(n3_df.index)

spd_counter = cdu_counter = fdp_counter = linke_counter = afd_counter = grüne_counter = fraktionslos_counter = 0 
parties_list = [spd_counter, cdu_counter, fdp_counter, linke_counter, afd_counter, grüne_counter, fraktionslos_counter]

def count_parties(party, my_list): 
    if party == "SPD":
        my_list[0] +=1
        return my_list
    elif party == "CDU/CSU":
        my_list[1] +=1
        return my_list
    elif party == "FDP":
        my_list[2] +=1
        return my_list
    elif party == "DIE LINKE":
        my_list[3] +=1
        return my_list
    elif party == "AfD":
        my_list[4] +=1
        return my_list
    elif bool(re.search("DIE GRÜNEN", str(party))) == True: 
        my_list[5] +=1
        return my_list
    elif party == 'fraktionslos' or party == 'Fraktionslos':
        my_list[6] +=1
        return my_list
    return


result = n3_df.apply(lambda row: count_parties(row['fraktion'], parties_list), axis=1)

         datum       redeID    vorname             nachname fraktion  \
60  11.10.2018  ID195506000       Marc             Bernhard      AfD   
61  11.10.2018  ID195506100  Christoph                 Ploß  CDU/CSU   
62  11.10.2018  ID195506200     Oliver               Luksic      FDP   
67  11.10.2018  ID195506700       Dirk              Spaniel      AfD   
68  11.10.2018  ID195506800       Rita  Schwarzelühr-Sutter      SPD   

                        name  \
60             Marc Bernhard   
61            Christoph Ploß   
62             Oliver Luksic   
67              Dirk Spaniel   
68  Rita Schwarzelühr-Sutter   

                                           inhalt_neu  word_count  \
60  herr präsident sehr geehrte kollegen es ist se...         598   
61  herr präsident verehrte kolleginnen und kolleg...         689   
62  herr präsident liebe kolleginnen und kollegen ...         835   
67  vielen dank sehr geehrte damen und herren sehr...         662   
68  sehr geehrter herr präsid

In [6]:
n4_df = dfc.loc[dfc['Anzahl Klimawandel'] != 0]
len(n4_df.index)

spd_counter = cdu_counter = fdp_counter = linke_counter = afd_counter = grüne_counter = fraktionslos_counter = 0 
parties_list = [spd_counter, cdu_counter, fdp_counter, linke_counter, afd_counter, grüne_counter, fraktionslos_counter]

result2 = n4_df.apply(lambda row: count_parties(row['fraktion'], parties_list), axis=1)

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
"""

fig, ax = plt.subplots(figsize = (25,17))    
fig = sns.barplot(x = "datum", y = "Anzahl Klima", data = dfc, 
                  estimator = sum, ci = None, ax=ax)

x_dates = pd.to_datetime(dfc['datum'], dayfirst=True).sort_values().unique()
ax.set_xticklabels(labels=x_dates, rotation=45, ha='right')

"""

'\n\nfig, ax = plt.subplots(figsize = (25,17))    \nfig = sns.barplot(x = "datum", y = "Anzahl Klima", data = dfc, \n                  estimator = sum, ci = None, ax=ax)\n\nx_dates = pd.to_datetime(dfc[\'datum\'], dayfirst=True).sort_values().unique()\nax.set_xticklabels(labels=x_dates, rotation=45, ha=\'right\')\n\n'

In [47]:
import gensim
import nltk
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

stop_words = stopwords.words('german')
manual_stop_words = (['werd','herr','kolleg','mehr','woll','wer', 'gut', 'wichtig','uber','konn','sag','frag', 'sag',
                      'frau', 'herr''schon', 'wurd', 'gibt', 'thema', 'ganz', 'mal', 'konn', 'glaub', 'gesagt', 'mach',
                      'geht', 'stell', 'all', 'and', 'red', 'hatt', 'debatt','mocht', 'dank', 'word', 'lieb', 'letzt',
                      'find', 'darub', 'darauf', 'desweg','eigent', 'vielleicht', 'genau', 'gar','bundesregier','deutsch', 'europa', 'mensch', 'deutschland', 'jahr', 
                     'antrag', 'sag', 'land', 'polit', 'minist', 'milliard', 'wurd', 'dam', 'uber', 'hat', 'ganz', 'prozent', 'mehr','braucht', 'frau', 'neu'])
stop_words = stop_words + manual_stop_words
stemmer = SnowballStemmer('german')
def tokenize_and_stem(text):
    
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        token = stemmer.stem(token)
        if len(token) > 2 and token not in stop_words:
            filtered_tokens.append(token)
    return filtered_tokens
long_string = n3_df.inhalt_neu.values.tolist()


In [48]:
import gensim.corpora as corpora
data_words = []
for speech in long_string:
    data_words.append(list(tokenize_and_stem(speech)))


In [49]:
id2word = corpora.Dictionary(data_words)
texts = data_words

In [50]:
corpus = [id2word.doc2bow(text) for text in texts]

In [52]:
from pprint import pprint
from gensim import models
num_topics = 25
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
#word2vec = models.Word2Vec(corpus)

In [53]:
lda_model = gensim.models.LdaMulticore(corpus=corpus_tfidf,
                                       id2word=id2word,
                                       num_topics=num_topics)

In [54]:
pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]

[(21,
  '0.000*"straftat" + 0.000*"kindeswohl" + 0.000*"roth" + 0.000*"lng-terminal" '
  '+ 0.000*"lng" + 0.000*"wohnungseigentumergemeinschaft" + 0.000*"geg" + '
  '0.000*"engagement" + 0.000*"rent" + 0.000*"migration"'),
 (13,
  '0.001*"jahressteuergesetz" + 0.000*"homeoffic" + 0.000*"ley" + '
  '0.000*"mietpreisbrems" + 0.000*"nachhalt" + 0.000*"union" + 0.000*"zitiert" '
  '+ 0.000*"miet" + 0.000*"offent" + 0.000*"wirtschaft"'),
 (12,
  '0.000*"euro" + 0.000*"landwirtschaft" + 0.000*"brauch" + 0.000*"leugn" + '
  '0.000*"ministerin" + 0.000*"landlich" + 0.000*"union" + '
  '0.000*"binnenschifffahrt" + 0.000*"klimaschutz" + 0.000*"wald"'),
 (11,
  '0.002*"euro" + 0.001*"haushalt" + 0.001*"landwirtschaft" + 0.001*"landwirt" '
  '+ 0.001*"deshalb" + 0.001*"ukrain" + 0.001*"massnahm" + 0.001*"million" + '
  '0.001*"gemeinsam" + 0.001*"kris"'),
 (3,
  '0.001*"wald" + 0.001*"indi" + 0.001*"landwirtschaft" + 0.001*"bau" + '
  '0.001*"kind" + 0.001*"hung" + 0.001*"wohnung" + 0.001*"union" 

In [55]:
import pyLDAvis
import pickle
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()

LDAvis_data_filepath =  os.path.join('./results/ldavis_prepared_'+str(num_topics))

In [56]:
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
   

/opt/tljh/user/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/opt/tljh/user/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/tljh/user/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/tljh/user/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/opt/tljh/user/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version

In [57]:
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.115785 -0.029871       1        1  67.471773
0      0.060866  0.058714       2        1  16.396342
3      0.023981 -0.002336       3        1   3.201047
20     0.017394 -0.000464       4        1   2.276170
7      0.013615 -0.004436       5        1   2.096136
15     0.014208 -0.004728       6        1   2.085558
6      0.019255  0.001771       7        1   2.078038
10     0.001807 -0.003426       8        1   0.832814
1      0.010616  0.000255       9        1   0.794320
2      0.005565 -0.005198      10        1   0.716484
8     -0.004439 -0.001688      11        1   0.597443
22     0.000600 -0.001416      12        1   0.529382
13    -0.009915 -0.000544      13        1   0.133514
9     -0.007699 -0.001353      14        1   0.121861
14    -0.016937 -0.002787      15        1   0.121723
18    -0.007636 -0.000664      16        1   0.112175
23    -0.018155 -0.001762      17        1   0.070619
21    -0.030412  0.001240      18        1   0.067910
16    -0.017430 -0.000985      19        1   0.062958
4     -0.022275 -0.000721      20        1   0.058768
5     -0.023298 -0.000270      21        1   0.046037
17    -0.027876 -0.001212      22        1   0.036527
19    -0.033220 -0.000699      23        1   0.034341
12    -0.026041  0.000682      24        1   0.031956
24    -0.038360  0.001898      25        1   0.026104, topic_info=                Term         Freq        Total Category  logprob  loglift
51              euro  2040.000000  2040.000000  Default  30.0000  30.0000
2617        haushalt  1739.000000  1739.000000  Default  29.0000  29.0000
2132  landwirtschaft  1382.000000  1382.000000  Default  28.0000  28.0000
232           brauch  1188.000000  1188.000000  Default  27.0000  27.0000
3521        landwirt  1256.000000  1256.000000  Default  26.0000  26.0000
...              ...          ...          ...      ...      ...      ...
726             wiss     0.074910   897.804645  Topic25  -8.5776  -1.1406
109         massnahm     0.076461  1235.006482  Topic25  -8.5571  -1.4390
294             haus     0.073801   772.123908  Topic25  -8.5925  -1.0047
51              euro     0.075525  2040.976088  Topic25  -8.5694  -1.9536
21          beispiel     0.072699   878.637105  Topic25  -8.6076  -1.1490

[1790 rows x 6 columns], token_table=       Topic      Freq          Term
term                                
11412      1  0.124192        -abgab
11412      2  0.838294        -abgab
11412      3  0.010349        -abgab
11412      7  0.010349        -abgab
39502      1  0.520883      -artikel
...      ...       ...           ...
15439      4  0.038285  ‑euro-ticket
15439      7  0.019142  ‑euro-ticket
15439      8  0.057427  ‑euro-ticket
15439      9  0.038285  ‑euro-ticket
15439     11  0.057427  ‑euro-ticket

[7608 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 1, 4, 21, 8, 16, 7, 11, 2, 3, 9, 23, 14, 10, 15, 19, 24, 22, 17, 5, 6, 18, 20, 13, 25])

In [44]:

from lda2vec import preprocess, Corpus

2022-06-17 16:26:54.602800: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-17 16:26:54.602863: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


ImportError: cannot import name 'preprocess' from 'lda2vec' (/opt/tljh/user/lib/python3.9/site-packages/lda2vec/__init__.py)